# Using Timelink in notebooks: tutorial

> First time use: follow instructions in the [README.md](README.md) file in this directory.

## Initial setup

Create a TimelinkNotebook object. This will be the interface for most of your interactions with Timelink.

The first time you run this notebook, you will be prompted to install ìpykernel. This is required to run Timelink in a notebook.

Note:
* First time run takes a little time as the required Docker images are downloaded.
* Timelink will default to using sqlite as the database, see [Receipts notebook for more control](1-receipts.ipynb)


In [1]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook()


TimelinkNotebook created a database to store data and a Kleio server to 
translate transcriptions in Kleio notation into data that can be imported
into a database.

To get information about the database and the Kleio server do ```tnlb.print_info()```

In [2]:
tlnb.print_info()

Project name: tutorial
Project home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial
Database type: sqlite
Database name: tutorial
Kleio image: timelinkserver/kleio-server
Kleio version: latest
Kleio server token: yNEupdoWvbKkAfALXitPVuC0D9PXUkPa
Kleio server URL: http://127.0.0.1:8089
Kleio server home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial
SQLite directory: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial/database/sqlite
TimelinkNotebook(project_name=tutorial, project_home=/Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial, db_type=sqlite, db_name=tutorial, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, postgres_version=latest)


### Database status

Count the number of rows in each table in the database.


In [3]:
tlnb.table_row_count_df()

,table,count
0,acts,3
1,attributes,14680
2,class_attributes,63
3,classes,13
4,entities,17182
5,geoentities,215
6,goods,0
7,kleiofiles,3
8,objects,0
9,persons,873


### Show the sources imported into the database

In [4]:
imported_files = tlnb.get_imported_files()
if len(imported_files) > 0:
    imported_files[["name","imported","nerrors","nwarnings"]].head()
else:
    print("No imported files")
imported_files

,path,name,structure,translator,translation_date,nerrors,nwarnings,error_rpt,warning_rpt,imported,imported_string
0,/kleio-home/kleio/auc-alunos.cli,auc-alunos.cli,/usr/local/timelink/clio/src/gacto2.str,gactoxml2.str,2024-02-04 08:37:10,0,0,No errors,No warnings,2024-02-04 08:38:54.291236,2024-02-04 08:38:54 UTC
1,/kleio-home/kleio/dehergne-locations-1644.cli,dehergne-locations-1644.cli,/usr/local/timelink/clio/src/gacto2.str,gactoxml2.str,2024-02-04 08:40:20,0,0,No errors,No warnings,2024-02-04 08:40:24.149131,2024-02-04 08:40:24 UTC
2,/kleio-home/kleio/dehergne-a.cli,dehergne-a.cli,/usr/local/timelink/clio/src/gacto2.str,gactoxml2.str,2024-02-04 14:08:26,0,0,No errors,No warnings,2024-02-04 14:08:36.475775,2024-02-04 14:08:36 UTC



# Update the database from Kleio sources.

Updates source translations and imports into database sources with no errors.

Only changed files since last import, or new files, will be processed.

In [5]:
import logging
logging.basicConfig(level=logging.INFO)

tlnb.update_from_sources()

## Check the status of the files

Check the import status of the translated files

I: Imported

E: Imported with error

W: Imported with warnings no errors

N: Not imported

U: Translation updated need to reimport

In [6]:
imported_files_df = tlnb.get_import_status()
imported_files_df[["import_status","import_errors","import_warnings","name","errors","warnings","path"]].sort_values("name")


,import_status,import_errors,import_warnings,name,errors,warnings,path
0,I,0,0,auc-alunos.cli,0,0,kleio/auc-alunos.cli
1,I,0,0,dehergne-a.cli,0,0,kleio/dehergne-a.cli
2,I,0,0,dehergne-locations-1644.cli,0,0,kleio/dehergne-locations-1644.cli


### Check the translation report

In [7]:
rpt = tlnb.get_translation_report(imported_files_df, 1)
print(rpt)


KleioTranslator - server version 12.2 - build 560 2024-01-08 08:18:15
4-2-2024 14-8

Processing data file dehergne-a.cli
-------------------------------------------
Generic Act translation module with geoentities (XML).
     Joaquim Ramos de Carvalho (joaquim@uc.pt) 
** New document: kleio
kleio translation started
Structure: gacto2.str
Prefix: 
Autorel: 
Translation count: 178
Obs: 
** Processing source fonte$dehergne-a
7: lista$dehergne-notices-a
*** End of File

Line 541 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-belchior-miguel-carneiro-leitao) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.
Line 814 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-jean-regis-lieou) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.
Line 876 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-alessandro-cicero) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.

Structure file: /usr/local/timelink/clio/src/gacto2.str
Structure processing report: /usr/local/timelink/clio/src/gacto2.srpt
Structure in JSON: /usr/loc

### Get the import report for a file

In case there are errors in the import phase check the import report.

In [8]:
rpt = tlnb.get_import_rpt(imported_files_df, rows=[0])
print(rpt)

No errors
No warnings


## Todo

This as data frame in a single function
* TimelinkNotebook.translate([files_df,paths=List[str], rows=List[int],status="T"])
* TimelinkNotebook.import([files_df,paths=List[str], rows=List[int],status="V"])



# Obter dados
---

# Getting data

### Procurar pessoa, mostrar em notação Kleio

---
### Search for people, show in Kleio notation

In [9]:
from timelink.api.models import Person

show_only=10

with tlnb.db.session() as session:
    persons = session.query(Person).filter(Person.name.like('%Abegão')).all()

    for person in persons[:show_only]:
        print(person.to_kleio())
        print()

n$Estevão de Matos Abegão/m/id=140338/obs="""
      """

                  Id: 140338
                  Código de referência: PT/AUC/ELU/UC-AUC/B/001-001/A/000002

                  Nome        : Estevão de Matos Abegão
                  Data inicial: 1728-10-01
                  Data final  : 1733-07-03
                  Filiação: Lourenço de Matos Pereira
                  Naturalidade: Elvas
                  Faculdade: Cânones

                  Matrícula(s): 01.10.1729
                  01.10.1730
                  01.10.1732

                  Instituta: 01.10.1728

                  Bacharel: 03.07.1733
              """
  """
  rel$function-in-act/n/auc-alumni-A-140337-140771/20200211
  atr$código-de-referência/""PT/AUC/ELU/UC-AUC/B/001-001/A/000002""/2021-05-17
  atr$data-do-registo/2021-05-17/2021-05-17
  atr$url/""https://pesquisa.auc.uc.pt/details?id=140338""/2021-05-17
  ls$uc.entrada/1728-10-01/1728-10-01
  ls$uc.saida/1733-07-03/1733-07-03
  ls$uc.entrada.ano/1728/1728-1

In [38]:
from sqlalchemy import select, and_
from sqlalchemy.orm import joinedload
from timelink.api.models import Person, Attribute

show_only=10

with tlnb.db.session() as session:
    stmt = select(Person).join(Person.attributes.and_(Attribute.the_type == 'naturalidade',
                                                      Attribute.the_value.like('Coimbra')))
    persons = session.execute(stmt).scalars().all()

    for person in persons[:show_only]:
        p = session.query(Person).filter(Person.id == person.id).options(joinedload(Person.attributes)).one()
        print(p.to_kleio())


140349
n$António de Aboim/m/id=140349/obs="""
      """

                  Id: 140349
                  Código de referência: PT/AUC/ELU/UC-AUC/B/001-001/A/000015

                  Nome        : António de Aboim
                  Data inicial: 1566-12-20
                  Data final  : 1574-07-24
                  Filiação: Heitor Fernandes

                  Naturalidade: Coimbra
                  Faculdade: Cânones

                  Matrícula(s): 07.10.1573
                  Instituta:
                  Bacharel: 20.07.1571
                  Formatura: 24.07.1574
                  Licenciado:
                  Doutor:

                  Outras informações:
                  Provas de curso: Cânones: 20.12.1566 até 02.06.1567
                  01.12.1567 até 15.05.1568
                  01.10.1568 até 31.06.1569
                  01.11.1569 até 31.07.1570
                  01.10.1570 até 18.06.1571
                  02.10.1572 até 30.06.1573
                  01.10.1573 até 10.07.15



###  Obter um dataframe a partir de atributos

---


###  Get a Dataframe from attributes


#### Exemplo: Faculdade, data de entrada e data de saída e grau dos naturais de Coimbra

In [39]:
from timelink.pandas import entities_with_attribute


# Get list of people with with a certain value in a specific attribute
df = entities_with_attribute(
                    the_type='naturalidade',
                    the_value='Coimbra',
                    person_info=True,
                    more_cols=['faculdade','uc.entrada','uc.saida','grau','grau.date'],
                    db=tlnb.db,
                    sql_echo=False)
df.info()

ImportError: cannot import name 'category_colors' from 'timelink.pandas.styles' (/Users/jrc/develop/timelink-py/timelink/pandas/styles.py)

In [12]:
df.head(5)

,name,sex,naturalidade,naturalidade.date,naturalidade.obs,faculdade,faculdade.date,faculdade.obs,uc.entrada,uc.entrada.date,uc.entrada.obs,uc.saida,uc.saida.date,uc.saida.obs,grau,grau.date,grau.obs
id,,,,,,,,,,,,,,,,,
140349,António de Aboim,m,Coimbra,20200211,None,Cânones,20200211,None,1566-12-20,20200211,None,1574-07-24,20200211,None,Bacharel,None,20.07.1571
140349,António de Aboim,m,Coimbra,20200211,None,Cânones,20200211,None,1566-12-20,20200211,None,1574-07-24,20200211,None,Formatura,None,24.07.1574
140367,Manuel de Vargas de Aboim,m,Coimbra,20200211,None,Cânones,20200211,None,0000-00-00,20200211,None,0000-00-00,20200211,None,NaN,None,NaN
140380,António de Abranches,m,Coimbra,20200211,None,Medicina,20200211,None,1733-10-01,20200211,None,1733-10-01,20200211,None,Bacharel em Artes,None,14.04.1733
140538,António de Abreu,m,Coimbra,20200211,None,Medicina,20200211,None,1670-10-01,20200211,None,1679-07-19,20200211,None,Bacharel em Artes,None,20.03.1671




###  Remover colunas sem valores

---



###  Remove empty columns

In [17]:
df.dropna(how='all', axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33 entries, 140349 to c1716-34-per1
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               33 non-null     object
 1   sex                33 non-null     object
 2   naturalidade       33 non-null     object
 3   naturalidade.date  33 non-null     object
 4   faculdade          26 non-null     object
 5   faculdade.date     26 non-null     object
 6   faculdade.obs      1 non-null      object
 7   uc.entrada         26 non-null     object
 8   uc.entrada.date    26 non-null     object
 9   uc.saida           26 non-null     object
 10  uc.saida.date      26 non-null     object
 11  grau               19 non-null     object
 12  grau.obs           18 non-null     object
 13  id                 33 non-null     object
dtypes: object(14)
memory usage: 3.9+ KB


In [18]:
df.head(5)

,name,sex,naturalidade,naturalidade.date,faculdade,faculdade.date,faculdade.obs,uc.entrada,uc.entrada.date,uc.saida,uc.saida.date,grau,grau.obs,id
id,,,,,,,,,,,,,,
140349,António de Aboim,m,Coimbra,20200211,Cânones,20200211,None,1566-12-20,20200211,1574-07-24,20200211,Bacharel,20.07.1571,140349
140349,António de Aboim,m,Coimbra,20200211,Cânones,20200211,None,1566-12-20,20200211,1574-07-24,20200211,Formatura,24.07.1574,140349
140367,Manuel de Vargas de Aboim,m,Coimbra,20200211,Cânones,20200211,None,0000-00-00,20200211,0000-00-00,20200211,NaN,NaN,140367
140380,António de Abranches,m,Coimbra,20200211,Medicina,20200211,None,1733-10-01,20200211,1733-10-01,20200211,Bacharel em Artes,14.04.1733,140380
140538,António de Abreu,m,Coimbra,20200211,Medicina,20200211,None,1670-10-01,20200211,1679-07-19,20200211,Bacharel em Artes,20.03.1671,140538



###  Remover as remissivas
---

###  Remove cross-references

In [20]:
no_crossref = df['uc.entrada'] != '0000-00-00'
df = df[no_crossref]

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, 140349 to c1716-34-per1
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               32 non-null     object
 1   sex                32 non-null     object
 2   naturalidade       32 non-null     object
 3   naturalidade.date  32 non-null     object
 4   faculdade          25 non-null     object
 5   faculdade.date     25 non-null     object
 6   faculdade.obs      1 non-null      object
 7   uc.entrada         25 non-null     object
 8   uc.entrada.date    25 non-null     object
 9   uc.saida           25 non-null     object
 10  uc.saida.date      25 non-null     object
 11  grau               19 non-null     object
 12  grau.obs           18 non-null     object
 13  id                 32 non-null     object
dtypes: object(14)
memory usage: 3.8+ KB


In [24]:
view_cols = ['name','naturalidade','faculdade','grau','uc.entrada','uc.saida']
df[view_cols].head(10)

,name,naturalidade,faculdade,grau,uc.entrada,uc.saida
id,,,,,,
140349,António de Aboim,Coimbra,Cânones,Bacharel,1566-12-20,1574-07-24
140349,António de Aboim,Coimbra,Cânones,Formatura,1566-12-20,1574-07-24
140380,António de Abranches,Coimbra,Medicina,Bacharel em Artes,1733-10-01,1733-10-01
140538,António de Abreu,Coimbra,Medicina,Bacharel em Artes,1670-10-01,1679-07-19
140538,António de Abreu,Coimbra,Medicina,Formatura,1670-10-01,1679-07-19
140538,António de Abreu,Coimbra,Medicina,Licenciado,1670-10-01,1679-07-19
140538,António de Abreu,Coimbra,Medicina,Doutor,1670-10-01,1679-07-19
140573,António Dionísio Pereira de Abreu,Coimbra,Medicina,Bacharel em Artes,1762-12-01,1771-07-31
140573,António Dionísio Pereira de Abreu,Coimbra,Medicina,Mestre,1762-12-01,1771-07-31



## Contagens

---

## Counting



###  Contagem de atributos a partir de uma tabela em memória

---

###  Count attributes from an existing dataframe



In [25]:
# create a column with the index values which are the id numbers
df['id'] = df.index.values

col = 'faculdade' # subotal by this column

# Use pandas groupby and specify unique value count for id
df_totals = df.groupby(col).agg({'id':'nunique',
                                                  'uc.entrada':'min',
                                                  'uc.saida':'max'})

df_totals.sort_values('id',ascending= False).head(30)

/var/folders/x7/qzytnd752559g0w475rfwkx00000gn/T/ipykernel_91143/3590605853.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] = df.index.values


,id,uc.entrada,uc.saida
faculdade,,,
Cânones,6,1566-12-20,1767-07-27
Medicina,5,1670-10-01,1823-10-20
Direito,1,1748-10-19,1748-10-19
Matemática,1,1868-10-02,1872-10-03



### Contagens na base de dados

Quando o atributo tem muitos valores e não é necessário
ter todas as pessoas em memória: contagem feita na base de dados

---

### Counting directly in the database
When there are many values and it is not
necessary to have all the people in memory:
count directly in the database.




In [29]:
from timelink.pandas import attribute_values

df_totals = attribute_values('grau',db=db)


In [30]:
df_totals.head(10)


,count,date_min,date_max
value,,,
Bacharel,171,20200211,20200211
Formatura,153,20200211,20200211
Licenciado,33,20200211,20200211
Bacharel em Artes,25,20200211,20200211
Doutor,11,20200211,20200211
Licenciado em Artes,5,20200211,20200211
Mestre,2,20200211,20200211


#### Filtrar por datas

Para evitar remissivas com data zero

---

#### Filter by dates

Avoid cross-references with zero date

##ERROR

In [31]:
df_totals = attribute_values('grau',dates_between=('1535','1919'),db=db)

In [32]:
df_totals.head(10)

,value,count,date_min,date_max
value,NaN,NaN,NaN,NaN


## Visualizar registos

---

## View records





### Visualizar uma pessoa

---

### View a person


#### Atributos de uma pessoa numa tabela, uma linha por attributo

---

#### Person attributes in a dataframe, one line per attribute

In [33]:
import pandas as pd
from timelink.pandas import group_attributes as person_attributes

pd.set_option('display.max_rows',1000)

id = '140349'
pdf = person_attributes([id],db=db)  # note id in a list
pdf[['date','type','value','attr_obs']].sort_values(['date','type'])

,date,type,value,attr_obs
id,,,,
140349,20200211,código-de-referência,"""PT/AUC/ELU/UC-AUC/B/001-001/A/000015""",None
140349,20200211,data-do-registo,2021-05-17,None
140349,20200211,exame,Exame e Grau de Bacharel em Cânones:,None
140349,20200211,faculdade,Cânones,None
140349,20200211,faculdade.ano,Cânones.1566,None
140349,20200211,grau,Bacharel,20.07.1571
140349,20200211,grau,Formatura,24.07.1574
140349,20200211,grau.ano,Bacharel.1571,20.07.1571
140349,20200211,grau.ano,Formatura.1574,24.07.1574


#### Atributos de uma pessoa numa tabela, attributos em colunas

---

#### Person attributes in a dataframe, attributes in columns

In [14]:
# Get list of people with with a certain value in a specific attribute

id = '316297'  # João Pedro Ribeiro
# id = '234295'  # Alexandre Metelo de Sousa
df = attribute_to_df(
                    the_type='uc-entrada',  # we need a base attribute
                    person_info=True,
                    more_cols=['instituta','matricula-faculdade','matricula-classe'],
                    filter_by=[id],
                    sql_echo=False)
view_cols = ['name','matricula-classe.date','matricula-classe','matricula-classe.obs']
df[view_cols].sort_values('matricula-classe.date')

,name,matricula-classe.date,matricula-classe,matricula-classe.obs
id,,,,
316297,João Pedro Ribeiro,1774-11-15,"Curso jurídico, 1º ano","Vol. III, L. I, fl. 20v."
316297,João Pedro Ribeiro,1774-11-15,"Curso jurídico, 1º ano","Vol. III, L. I, fl. 20v."
316297,João Pedro Ribeiro,1774-11-15,"Curso jurídico, 1º ano","Vol. III, L. I, fl. 20v."
316297,João Pedro Ribeiro,1774-11-15,"Curso jurídico, 1º ano","Vol. III, L. I, fl. 20v."
316297,João Pedro Ribeiro,1774-11-15,"Curso jurídico, 1º ano","Vol. III, L. I, fl. 20v."
316297,João Pedro Ribeiro,1774-11-15,"Curso jurídico, 1º ano","Vol. III, L. I, fl. 20v."
316297,João Pedro Ribeiro,1774-11-15,"Curso jurídico, 1º ano","Vol. III, L. I, fl. 20v."
316297,João Pedro Ribeiro,1775-00-00,"Matemática, 1º ano","Vol. IV, L. 4, fl. 48"
316297,João Pedro Ribeiro,1775-00-00,"Curso jurídico, 2º ano","Vol. IV, L. I, fl. 48"


#### Examinar potenciais duplicados

---

#### Examine potentital duplicates

In [15]:
from timelink.notebooks.pandas import display_group_attributes
pd.set_option('display.max_rows',250)

no_show=['código-de-referência','data-do-registo','url','faculdade.ano','naturalidade.ano',
         'matricula-faculdade.ano','nome-apelido','nome-primeiro','nome-geografico.ano',
         'grau.ano','matricula-outra.ano','nome-geografico','instituta.ano']

dup_ids = ['234295','234710',]  # Alexandre Metelo de

display_group_attributes(dup_ids,
                             header_cols=['uc-entrada','naturalidade','faculdade','nome-pai'],
                             exclude_attributes=no_show,
                             sort_attributes=['date','type','value'],
                             cmap_name='Pastel1')

,id,uc-entrada,naturalidade,faculdade,nome-pai
0,234295,1704-11-07,Marialva,Cânones,nan
1,234710,1705-10-24,Marialva,Matemática,Manuel Cardoso Metelo


,date,id,type,value,attr_obs
0,1704-11-07,234295,faculdade,Cânones,Cânones
1,1704-11-07,234295,instituta,1704-11-07,07.11.1704 1704-11-07
2,1704-11-07,234295,naturalidade,Marialva,None
3,1704-11-07,234295,nome,Alexandre Metelo de Sousa,None
4,1704-11-07,234295,uc-entrada,1704-11-07,None
5,1704-11-07,234295,uc-entrada.ano,1704,None
6,1705-10-24,234710,faculdade,Matemática,Matemática
7,1705-10-24,234295,matricula-faculdade,Cânones,24.10.1705
8,1705-10-24,234710,matricula-faculdade,Matemática,24.10.1705
9,1705-10-24,234710,naturalidade,Marialva,None


#### Notação Kleio

Ver [Kleio notation](README_kleio.md) [EN]

---

#### Kleio notation

See [Kleio notation](README_kleio.md)

In [5]:
from ucalumni.aluno import get_and_process_aluno
from ucalumni.mapping import map_aluno_kperson

from ucalumni.extractors import get_extractors

get_extractors()

id =  '234295'  #'316297'

aluno = get_and_process_aluno(id,db)
kaluno = map_aluno_kperson(aluno)
kleio = kaluno.to_kleio()

print(kleio)

n$Alexandre Metelo de Sousa/m/id=234295/obs="""

      Id: 234295
      Código de referência: PT/AUC/ELU/UC-AUC/B/001-001/S/010952

      Nome        : Alexandre Metelo de Sousa
      Data inicial: 1704-11-07
      Data final  : 1706-10-01
      Filiação:
      Naturalidade: Marialva
      Faculdade: Cânones

      Matrícula(s): 24.10.1705
      01.10.1706

      Instituta: 07.11.1704
  """
 atr$código-de-referência/"PT/AUC/ELU/UC-AUC/B/001-001/S/010952"/2022-06-08
 atr$data-do-registo/2022-06-08/2022-06-08
 atr$url/"http://pesquisa.auc.uc.pt/details?id=234295"/2022-06-08
 ls$uc-entrada/1704-11-07/1704-11-07
 ls$uc-saida/1706-10-01/1706-10-01
 ls$uc-entrada.ano/1704/1704-11-07
 ls$uc-saida.ano/1706/1706-10-01
 ls$nome/Alexandre Metelo de Sousa/1704-11-07
 ls$nome-primeiro/Alexandre/1704-11-07
 ls$nome-apelido/Metelo de Sousa/1704-11-07
 ls$nome-apelido/Sousa/1704-11-07
 ls$nome-geografico/Marialva/1704-11-07
 ls$nome-geografico.ano/Marialva.1704/1704-11-07
 ls$naturalidade/Marialva/170

#### Notação Kleio directamente da base de dados

Ver [Kleio notation](README_kleio.md) [EN]

---

#### Kleio notation directly from database

See [Kleio notation](README_kleio.md)

In [11]:
from ucalumni.config import Session
from timelink.mhk.models.person import Person

with Session(bind=db.get_engine()) as session:

    p: Person = session.query(Person).order_by(Person.id).first()
    k = p.to_kleio()
    print(p.to_kleio())


n$Manuel Barreto Cabaça/m/id=127764/obs="""

      Id: 127764
      Código de referência: PT/AUC/ELU/UC-AUC/B/001-001/C/000001

      Nome        : Manuel Barreto Cabaça
      Data inicial: 1690-10-01
      Data final  : 1697-10-01
      Filiação: Manuel Barreto
      Naturalidade: Portalegre

      Faculdade: Cânones
      Instituta:01.10.1690
      Matrícula(s): 01.10.1690
      01.10.1691
      01.10.1692
      01.10.1693
      01.10.1694
      01.10.1695
      01.10.1696
      01.10.1697

      Bacharel - 26.05.1696
      Formatura - 20.11.1697
  """
  rel$parentesco/filho/Manuel Barreto/127764-pai/1690-10-01
  rel$function-in-act/n/C/
  atr$código-de-referência/"PT/AUC/ELU/UC-AUC/B/001-001/C/000001"/2021-05-13
  atr$data-do-registo/2021-05-13/2021-05-13
  atr$url/"https://pesquisa.auc.uc.pt/details?id=127764"/2021-05-13
  ls$uc-entrada/1690-10-01/1690-10-01
  ls$uc-saida/1697-10-01/1697-10-01
  ls$uc-entrada.ano/1690/1690-10-01
  ls$uc-saida.ano/1697/1697-10-01
  ls$nome/Manuel Ba

# Debug

In [5]:
import requests

kurl2 = f"{url}/rest/exports/reference_sources/varia/auc-alunos-264605-A-140337-140771.xml"

print(kurl2)
headers = {"Authorization": f"Bearer {token}"}

response = requests.get(kurl2, headers=headers)
if response.status_code == 200:
    xml_string = response.text
    print(xml_string[0:2048])
else:
    print(f"Error: {response.status_code} - {response.reason}")

http://127.0.0.1:8090/rest/exports/reference_sources/varia/auc-alunos-264605-A-140337-140771.xml
<?xml version='1.0'?>
<KLEIO STRUCTURE="/usr/local/timelink/clio/src/gacto2.str" SOURCE="/kleio-home/reference_sources/varia/auc-alunos-264605-A-140337-140771.cli" TRANSLATOR="gactoxml2.str" WHEN="2023-10-17 4:28:16" OBS="" SPACE="">
<CLASS NAME="source" SUPER="entity" TABLE="sources" GROUP="fonte">
     <ATTRIBUTE NAME="id" COLUMN="id" CLASS="id" TYPE="varchar" SIZE="64" PRECISION="0" PKEY="1" ></ATTRIBUTE>
     <ATTRIBUTE NAME="date" COLUMN="the_date" CLASS="date" TYPE="varchar" SIZE="24" PRECISION="0" PKEY="0" ></ATTRIBUTE>
     <ATTRIBUTE NAME="type" COLUMN="the_type" CLASS="type" TYPE="varchar" SIZE="32" PRECISION="0" PKEY="0" ></ATTRIBUTE>
     <ATTRIBUTE NAME="value" COLUMN="the_value" CLASS="value" TYPE="varchar" SIZE="254" PRECISION="0" PKEY="0" ></ATTRIBUTE>
     <ATTRIBUTE NAME="loc" COLUMN="loc" CLASS="loc" TYPE="varchar" SIZE="64" PRECISION="0" PKEY="0" ></ATTRIBUTE>
     <ATTR

In [29]:
from timelink.kleio.importer import import_from_xml

file = "https://raw.githubusercontent.com/time-link/timelink-py/f76007cb7b98b39b22be8b70b3b2a62e7ae0c12f/tests/xml_data/b1685.xml"  # noqa

with db.session() as session:
    stats = import_from_xml(file, session=session,options={"return_stats": True})
stats


Storing 2 postponed relations


{'datetime': 1697549928.160403,
 'machine': 'jrc-air-m2.local',
 'database': sqlite:////Users/jrc/develop/timelink-py/tests/timelink-home/sources/test-project/database/sqlite/timelink.db.sqlite,
 'file': 'https://raw.githubusercontent.com/time-link/timelink-py/f76007cb7b98b39b22be8b70b3b2a62e7ae0c12f/tests/xml_data/b1685.xml',
 'import_time_seconds': 2.4436087608337402,
 'entities_processed': 747,
 'entity_rate': 305.69541735688057,
 'person_rate': 78.98154692085402,
 'nerrors': 0,
 'errors': []}